In [14]:
# %pip install gym_super_mario_bros==7.3.0 nes_py

In [15]:
import numpy as np
import pandas as pd

In [16]:
#import the game
import gym_super_mario_bros
#import the joypad wrapper
from nes_py.wrappers import JoypadSpace
#import the simplified controls
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

In [17]:
# from nes_py import NESEnv
# _reset = NESEnv.reset

# def reset(*args, **kwargs):
#     obs_info = _reset(*args, **kwargs)
#     obs, info = obs_info if type(obs_info) == tuple else (obs_info, {})
#     return obs, info

# NESEnv.reset = reset

In [18]:
# #setup the environment
# env=gym_super_mario_bros.make('SuperMarioBros-v0',apply_api_compatibility=True,render_mode='human')
# env=JoypadSpace(env,SIMPLE_MOVEMENT)

In [19]:
# #create restart flag
# done=True
# #loop through each frame
# for step in range(10000):
#     if done:
#         #start the game
#         env.reset()
#     #do random actions
#     state, reward, done, truncated, info=env.step(env.action_space.sample())
#     #show the game
#     env.render()
# #close the game
# env.close()

In [20]:
# %pip install torch torchvision torchaudio

In [21]:
# %pip install stable-baselines3[extra]

In [22]:
#import framestacker and grayscaler
from gym.wrappers import GrayScaleObservation
#import vectorization wrappers
from stable_baselines3.common.vec_env import VecFrameStack,DummyVecEnv
#import matplotlib to show framestacking
from matplotlib import pyplot as plt

In [32]:
JoypadSpace.reset = lambda self, **kwargs: self.env.reset(**kwargs)

#1.Create base environment
env=gym_super_mario_bros.make('SuperMarioBros-v0',apply_api_compatibility=True,render_mode='human')
#2.Simplify Controls
env=JoypadSpace(env,SIMPLE_MOVEMENT)
#3.Grayscale
env=GrayScaleObservation(env,keep_dim=True)
#4.Wrap inside Dummy env
env=DummyVecEnv([lambda:env])
#5.Stack the frames
env=VecFrameStack(env,4,channels_order='last')

In [33]:
env.reset()

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

In [25]:
plt.imshow(state)

NameError: name 'state' is not defined

In [ ]:
env.close()